Tutaj ja namagam się przetworzyć dane tekstowe w kształt macierzy (c) Ivan

In [1]:
import numpy as np
import pandas as pd
import nltk
from matplotlib import pyplot as plt
from tqdm import tqdm

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.decomposition import PCA
from statsmodels.graphics.gofplots import qqplot
from scipy.stats import norm, uniform
from pywsd.utils import lemmatize_sentence

pd.set_option('display.max_columns', None)
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')


Warming up PyWSD (takes ~10 secs)... took 2.759860038757324 secs.
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ivansmaliakou/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/ivansmaliakou/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
df = pd.read_csv('../data/processed_data/processed_data.csv')

In [3]:
text_data = df.loc[:, df.dtypes == 'object']

In [4]:
narrative = df['NARRATIVE'].values
narrative

array(['THE RELEASE OCCURRED NEAR A NORMALLY DRY DRAINAGE DITCH THAT ONLY FLOWS DURING HEAVY RAINY CONDITIONS. IT WAS IDENTIFIED BY THE LAND OWNER WHO NOTIFIED THE OPERATOR.  THE REPAIRS HAVE BEEN MADE AND CLEANED UP. THE CAUSE OF THE LEAK IS BELIEVED TO HAVE BEEN EXTERNAL CORROSION AT ANY ABOVE GROUND EXPOSURE.',
       "ON DECEMBER 30, 2022, A CENTURION PIPELINE, L.P. (CPL) EMPLOYEE DISCOVERED A LEAK ON A 2'' UNION. THE CPL EMPLOYEE ARRIVED ON-SITE AT 08:35 AM AND CONFIRMED THE LEAK WAS INSIDE CPL'S MIDKIFF STATION LOCATED NEAR MIDLAND, IN MIDLAND COUNTY, TEXAS. THREE (3.4) BARRELS OF CRUDE OIL WERE RELEASED AND RECOVERED DURING THE PROCESS. CPL REMEDIATED THE CONTAMINATED SOIL ON SITE. NO INJURIES, FATALITIES OR FIRES WERE ASSOCIATED WITH THIS INCIDENT. CPL'S INVESTIGATION DETERMINED THAT THAT CROSS THREADING OCCURRED WHEN THE NUT WAS THREADED ONTO THE UNION BODY. THERE WAS VISIBLE DAMAGE TO THE THREADS ON THE MALE END OF THE BODY INDICATING THAT CROSS THREADING OCCURRED DUE TO MISA

In [5]:
df['NARRATIVE'] = df['NARRATIVE'].fillna('')
narrative = df['NARRATIVE']

In [6]:
narrative_sentences = []
for raw_sent in tqdm(narrative):
    narrative_sentences.append(lemmatize_sentence(raw_sent))

100%|██████████| 8134/8134 [03:01<00:00, 44.87it/s]


In [7]:
for i in range(len(narrative_sentences)):
    narrative_sentences[i] = ''.join(narrative_sentences[i])

In [8]:
tfidfvectorizer = TfidfVectorizer(analyzer='word',stop_words= 'english')
tfidf_wm = tfidfvectorizer.fit_transform(narrative_sentences)
tfidf_tokens = tfidfvectorizer.get_feature_names_out()
df_tfidfvect = pd.DataFrame(data = tfidf_wm.toarray(),columns = tfidf_tokens)

In [16]:
pca = PCA(svd_solver='randomized', n_components=500)
reduced_tfidf = pca.fit_transform(df_tfidfvect)

In [30]:
column_names = [f'narrative_tfidf-{i}' for i in range(500)]
tfidf = pd.DataFrame(reduced_tfidf, columns=column_names)
joined_df = df.join(tfidf)

In [32]:
joined_df.to_csv('../data/processed_data/processed_data_tf_idf.ivan.csv')